In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving test.csv to test.csv
Saving train.csv to train.csv
User uploaded file "test.csv" with length 420783 bytes
User uploaded file "train.csv" with length 987712 bytes


In [1]:
!ls


sample_data  test.csv  train.csv


In [2]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.7.0
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


In [3]:
import pandas as pd
train_data = pd.read_csv('train.csv')

In [4]:
train_np = train_data.to_numpy()

In [5]:
test_data = pd.read_csv('test.csv')
test_np = test_data.to_numpy()

In [6]:
!pip install transformers

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [8]:
batch_1 = train_data[:2000]
test_batch = test_data[:2000]
test_batch2 = test_data[2000:]

In [11]:
train_data.head()
test_batch.tail()

,id,keyword,location,text
1995,6720,lava,somewhere over the rainbow,I lava you
1996,6728,lava,NaN,@YoungHeroesID \n4. LAVA BLAST dan POWER RED #...
1997,6732,lava,"Los Angeles, CA",When the game tells you to stop going to the N...
1998,6733,lava,NaN,White Pink Lava Bracelet10mm Lava Beads Bracel...
1999,6739,lava,NaN,@_Souuul * gains super powers im now lava girl...


In [12]:
batch_1.target.value_counts()


0    1246
1     754
Name: target, dtype: int64

In [13]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
tokenized = batch_1['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [14]:
test_tokenized = test_batch['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
test_tokenized2 = test_batch2['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [15]:
max_len = 0
for i in test_tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded_test = np.array([i + [0]*(max_len-len(i)) for i in test_tokenized.values])

In [16]:
max_len = 0
for i in test_tokenized2.values:
    if len(i) > max_len:
        max_len = len(i)

padded_test2 = np.array([i + [0]*(max_len-len(i)) for i in test_tokenized2.values])

In [23]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [24]:
np.array(padded_test).shape

(2000, 70)

In [25]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 82)

In [17]:
attention_mask_test = np.where(padded_test != 0, 1, 0)
attention_mask_test.shape

(2000, 70)

In [18]:
attention_mask_test2 = np.where(padded_test2 != 0, 1, 0)
attention_mask_test2.shape

(1263, 73)

In [26]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [19]:
test_input_ids = torch.tensor(padded_test)  
attention_mask_test = torch.tensor(attention_mask_test)

with torch.no_grad():
    test_last_hidden_states = model(test_input_ids, attention_mask=attention_mask_test)

In [20]:
test_input_ids2 = torch.tensor(padded_test2)  
attention_mask_test2 = torch.tensor(attention_mask_test2)

with torch.no_grad():
    test_last_hidden_states2 = model(test_input_ids2, attention_mask=attention_mask_test2)

In [27]:
features = last_hidden_states[0][:,0,:].numpy()

In [21]:
test_features = test_last_hidden_states[0][:,0,:].numpy()
test_features2 = test_last_hidden_states2[0][:,0,:].numpy()

In [28]:
labels = batch_1.target

In [29]:
X_train, X_test, y_train, y_test = train_test_split(features, labels) 

In [30]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(X_train, y_train)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947}
best scrores:  0.7806666666666666


In [31]:
lr_clf = LogisticRegression(C=5.3)
lr_clf.fit(X_train, y_train)

LogisticRegression(C=5.3)

In [32]:
lr_clf.score(X_test, y_test)

0.792

In [33]:
lr_pred = lr_clf.predict(test_features)
lr_pred2 = lr_clf.predict(test_features2)

In [42]:
lr_pred.shape

(2000,)

In [38]:
lr_pred_final = np.append(lr_pred, lr_pred2)

In [43]:
lr_pred_final.shape

(3263,)

In [47]:
distilBertColab = pd.DataFrame({'id':test_data.id, 'target':lr_pred_final})
#path = '/Users/jamester/Documents/MachineLearning/KaggleComps/nlp-getting-started/distilBertColab'
distilBertColab.to_csv('distilBertColab.csv', index=False)
#distilBertColab.download

In [ ]:
from google.colab import drive
drive.mount('/content/drive')